`Sequential Chain`

In [1]:
from dotenv import load_dotenv
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda
from langchain_groq import ChatGroq

load_dotenv()

llm = ChatGroq(
    model="llama3-8b-8192",
    temperature=0.5,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a music historian focusing on Nigerian music, especially afrobeats. Provide accurate information about {music_topic}."),
        ("human", "Please share {fact_count} interesting facts."),
    ]
)

remove_extra_spaces = RunnableLambda(lambda x: " ".join(x.split()))

summarize_text = RunnableLambda(lambda x: f"SUMMARY: {x[:80]}... [shortened]\n\n{x}")

chain = (
    prompt_template 
    | llm 
    | StrOutputParser() 
    | remove_extra_spaces 
    | summarize_text
)

result = chain.invoke({"music_topic": "afrobeats", "fact_count": 3})

print(result)

SUMMARY: I'd be delighted to share some fascinating facts about Afrobeats! Here are three... [shortened]

I'd be delighted to share some fascinating facts about Afrobeats! Here are three interesting ones: 1. **Afrobeats is a fusion of traditional Yoruba music and American funk**: Afrobeats originated in the 1960s in Nigeria, particularly in the city of Lagos. It's a fusion of traditional Yoruba music, Highlife, and American funk, soul, and jazz. The genre was heavily influenced by the likes of Fela Kuti, who is often referred to as the "father of Afrobeats." Fela's music combined traditional Yoruba rhythms with American funk and soul, creating a unique sound that resonated with the Nigerian people. 2. **Afrobeats is a global phenomenon**: Afrobeats has transcended its Nigerian roots and has become a global phenomenon. The genre has spread to various parts of the world, including the UK, Ghana, South Africa, and the United States. Artists like Wizkid, Davido, and Tiwa Savage have gained

`Parallel Chain`

In [2]:
from dotenv import load_dotenv
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableParallel, RunnableLambda
from langchain_groq import ChatGroq

load_dotenv()

llm = ChatGroq(
    model="llama3-8b-8192",
    temperature=0.5,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are an expert in Nigerian Afrobeat music."),
        (
            "human",
            "List the main highlights of {music_topic}. "
            "Focus on historical origins, cultural influences, key artists, "
            "and anything else that stands out.",
        ),
    ]
)

def create_impact_prompt(highlights: str):
    """Prompt for analyzing the positive impact based on the highlights."""
    impact_template = ChatPromptTemplate.from_messages(
        [
            ("system", "You are an expert in Nigerian Afrobeat music."),
            (
                "human",
                "Given these highlights about Afrobeat:\n\n{highlights}\n\n"
                "List the positive impacts these highlights have had "
                "on the Nigerian music scene, cultural identity, or global influence.",
            ),
        ]
    )
    return impact_template.format_prompt(highlights=highlights)

def create_challenges_prompt(highlights: str):
    """Prompt for analyzing the challenges or controversies based on the highlights."""
    challenges_template = ChatPromptTemplate.from_messages(
        [
            ("system", "You are an expert in Nigerian Afrobeat music."),
            (
                "human",
                "Given these highlights about Afrobeat:\n\n{highlights}\n\n"
                "Discuss any challenges, controversies, or criticisms that might arise "
                "from these highlights—such as commercialization, cultural misappropriation, "
                "or stylistic disputes.",
            ),
        ]
    )
    return challenges_template.format_prompt(highlights=highlights)

def combine_impact_challenges(impact: str, challenges: str):
    """Combine the positive impacts and potential challenges into a final response."""
    return (
        "POSITIVE IMPACTS:\n"
        f"{impact}\n\n"
        "POTENTIAL CHALLENGES:\n"
        f"{challenges}"
    )

impact_branch_chain = (
    RunnableLambda(lambda x: create_impact_prompt(x)) 
    | llm 
    | StrOutputParser()
)

challenges_branch_chain = (
    RunnableLambda(lambda x: create_challenges_prompt(x)) 
    | llm 
    | StrOutputParser()
)

chain = (
    prompt_template
    | llm
    | StrOutputParser()
    | RunnableParallel(branches={"impact": impact_branch_chain, "challenges": challenges_branch_chain})
    | RunnableLambda(lambda x: combine_impact_challenges(x["branches"]["impact"], x["branches"]["challenges"]))
)

result = chain.invoke({"music_topic": "Fela Kuti's Afrobeat legacy"})

print(result)

POSITIVE IMPACTS:
Based on the highlights, here are the positive impacts of Fela Kuti's Afrobeat legacy:

**Nigerian Music Scene:**

1. Paved the way for future generations of Afrobeat musicians, inspiring artists like Wizkid, Tiwa Savage, and Burna Boy.
2. Influenced the development of other genres, such as juju music, and blended them with Afrobeat to create unique sounds.
3. Established Afrobeat as a distinct genre, setting the stage for future Nigerian musicians to explore and experiment with the style.

**Cultural Identity:**

1. Celebrated and preserved Nigerian cultural heritage, particularly the Yoruba language, proverbs, and mythology.
2. Used music as a means of storytelling, sharing Nigerian experiences, and addressing social and political issues.
3. Provided a platform for Nigerian artists to express themselves and connect with their cultural roots.

**Global Influence:**

1. Introduced Afrobeat to a global audience, paving the way for other African musicians to break into 

`Branched Chains`

In [8]:
from dotenv import load_dotenv
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableBranch
from langchain_groq import ChatGroq

load_dotenv()

# Create the LLM
llm = ChatGroq(
    model="llama3-8b-8192",
    temperature=0.5,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

classification_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant specialized in Nigerian Afrobeat music management."),
        ("human",
         "Classify this feedback regarding the Afrobeat performance as 'positive', 'negative', 'neutral', or 'escalate': {feedback}."),
    ]
)

positive_feedback_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are the manager for a top-tier Nigerian Afrobeat band, eager to provide gracious responses to fans."),
        ("human", 
         "Generate a thank-you note for this positive feedback about our recent Afrobeat performance: {feedback}."),
    ]
)

negative_feedback_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are the manager for a top-tier Nigerian Afrobeat band, striving to address concerns sincerely."),
        ("human", 
         "Generate a response that acknowledges and addresses this negative feedback related to the Afrobeat show: {feedback}."),
    ]
)

neutral_feedback_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are the manager for a top-tier Nigerian Afrobeat band, always seeking more context to improve fan satisfaction."),
        ("human",
         "This feedback is somewhat neutral regarding our Afrobeat music. Request more details to help us improve: {feedback}."),
    ]
)

escalate_feedback_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are the manager for a top-tier Nigerian Afrobeat band, sometimes needing external support for complex situations."),
        ("human",
         "Generate a message indicating that this feedback about the Afrobeat performance is critical and needs escalation to a senior manager: {feedback}."),
    ]
)

branches = RunnableBranch(
    (
        lambda x: "positive" in x,
        positive_feedback_template | llm | StrOutputParser()
    ),
    (
        lambda x: "negative" in x,
        negative_feedback_template | llm | StrOutputParser()
    ),
    (
        lambda x: "neutral" in x,
        neutral_feedback_template | llm | StrOutputParser()
    ),
    escalate_feedback_template | llm | StrOutputParser()
)

classification_chain = classification_template | llm | StrOutputParser()

chain = classification_chain | branches

review = "Your show was so great"
result = chain.invoke({"feedback": review})

print(result)


Dear [Name],

We are over the moon to hear that you thoroughly enjoyed our recent Afrobeat performance! Your kind words have left us beaming with joy and gratefulness. We are thrilled to know that our music was able to bring a smile to your face and get you moving to the rhythm.

We would like to extend our sincerest gratitude for taking the time to share your thoughts with us. Your feedback means the world to us, and we are thrilled that you felt our performance was "great". We strive to deliver high-energy shows that leave a lasting impression on our audience, and it's wonderful to know that we were able to achieve that for you.

Please know that you are an integral part of our Afrobeat family, and we are honored to have you as part of our community. We can't wait to share more of our music and energy with you at our next show!

Thank you once again for your wonderful feedback. We look forward to seeing you on the dance floor soon!

Warm regards,
[Your Band Name]
